In [14]:
import sys
print(sys.executable)


/Users/ezgimburcakakseki/Desktop/BigData/.venv/bin/python


In [ ]:
!pip install scipy
!pip install pandas

In [ ]:
import os
import gzip
import shutil
import requests
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from itertools import combinations
from tqdm import tqdm
from pptx import Presentation
from pptx.util import Inches
import random
import pandas as pd
from math import comb
import matplotlib.patches as mpatches


ModuleNotFoundError: No module named 'requests'

In [ ]:
import os
print(os.getcwd())  # shows where Python is looking for files


In [ ]:
#Loading the dataset
dataset_file = "/Users/ezgimburcakakseki/Desktop/BigData/facebook_combined.txt"
  # Make sure this file is in the working directory

G = nx.Graph()

print("Loading graph...")
with open(dataset_file) as f:
    for line in tqdm(f):
        u, v = map(int, line.strip().split())
        G.add_edge(u, v)

print(f"Graph loaded: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

In [ ]:
# 1.Graph Motif: Triangles
#Triangle participation per node (how many triangles each user is part of)
triangles_per_node = nx.triangles(G)
total_triangles = sum(triangles_per_node.values()) // 3
print(f"Total triangles in the network: {total_triangles}")
plt.figure(figsize=(8, 5))
plt.hist(triangles_per_node.values(), bins=20, color='skyblue', edgecolor='black')
plt.title("Distribution of Triangle Participation")
plt.xlabel("Number of triangles")
plt.ylabel("Number of nodes")
# Save the figure as an image
plt.savefig("triangle_participation_distribution.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 2.Local motif roles (triangles + stars)
# reveals the roles of nodes: whether they are connectors/bridges, part of dense cliques, or a mix.

motif_roles = {}
for node in G.nodes():
    deg = G.degree(node)
    tri = triangles_per_node[node]
    # 2-star motifs: node connected to 2 neighbors who are not connected
    two_stars = deg * (deg - 1) // 2 - tri
    motif_roles[node] = {"triangles": tri, "two_stars": two_stars}

# Visualize triangle vs star participation
triangle_counts = [v["triangles"] for v in motif_roles.values()]
two_star_counts = [v["two_stars"] for v in motif_roles.values()]

plt.figure(figsize=(8, 5))
plt.scatter(two_star_counts, triangle_counts, alpha=0.6)
plt.xlabel("2-star motifs (loose connections)")
plt.ylabel("Triangle motifs (tight connections)")
plt.title("Local Motif Roles")
plt.savefig("local_motif_roles.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
#Most Connected Nodes
most_connected = sorted(G.degree, key=lambda x: x[1], reverse=True)[:10]
print("Top 10 most connected nodes:", most_connected)

In [ ]:
# 3.Categorizing nodes by triangle participation and visualizes a sample of the network,
# Define "core" as nodes in many triangles, "periphery" as nodes in few/no triangles
# -----------------------------
# Create motif_roles dictionary
motif_roles = {}
for node in G.nodes():
    deg = G.degree(node)
    tri = triangles_per_node[node]
    # 2-star motifs: node connected to 2 neighbors who are not connected
    two_stars = deg * (deg - 1) // 2 - tri
    motif_roles[node] = {"triangles": tri, "two_stars": two_stars}

# Assign node roles (core/intermediate/periphery)
triangle_values = np.array([v["triangles"] for v in motif_roles.values()])
threshold_core = np.percentile(triangle_values, 90)
threshold_periphery = np.percentile(triangle_values, 10)

roles = {}
for node, counts in motif_roles.items():
    if counts["triangles"] >= threshold_core:
        roles[node] = "core"
    elif counts["triangles"] <= threshold_periphery:
        roles[node] = "periphery"
    else:
        roles[node] = "intermediate"

# Count nodes in each role
role_counts = Counter(roles.values())
print("Node roles:", role_counts)

# 4. Bar chart: number of nodes per role
plt.figure(figsize=(6,4))
plt.bar(role_counts.keys(), role_counts.values(), color=['red', 'orange', 'blue'])
plt.title("Core vs Intermediate vs Periphery Node Counts")
plt.ylabel("Number of nodes")
plt.savefig("core_intermediate_periphery_nodes.png", dpi=300, bbox_inches='tight')
plt.show()

# 5. Node-level network visualization
# Node colors based on role
role_colors = {"core": "red", "intermediate": "orange", "periphery": "blue"}
node_colors = [role_colors[roles[node]] for node in G.nodes()]

# Fixed, smaller node sizes for clarity
node_sizes = [20 + np.log1p(motif_roles[node]["triangles"]) * 10 for node in G.nodes()]

# Spring layout for all nodes
pos = nx.spring_layout(G, seed=42, k=0.15)

plt.figure(figsize=(14,14))
nx.draw_networkx_nodes(
    G,
    pos,
    node_color=node_colors,
    node_size=node_sizes,
    alpha=0.7
)
nx.draw_networkx_edges(
    G,
    pos,
    alpha=0.1,  # very transparent edges for clarity
    edge_color='gray'
)

# Add legend
red_patch = mpatches.Patch(color='red', label='Core')
orange_patch = mpatches.Patch(color='orange', label='Intermediate')
blue_patch = mpatches.Patch(color='blue', label='Periphery')
plt.legend(handles=[red_patch, orange_patch, blue_patch], loc='upper right')

plt.title("Facebook Network: Node Roles (Core=Red, Intermediate=Orange, Periphery=Blue)")
plt.axis('off')
plt.savefig("Node_Roles_Network.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 4.Degree Distribution Across Network - STAR MOTIFS 
degrees = [G.degree(n) for n in G.nodes()]

print(f"Max star size (max degree): {max(degrees)}")

plt.figure(figsize=(8,5))
plt.hist(degrees, bins=40, edgecolor='black')
plt.title("Degree Distribution across Facebook Network")
plt.xlabel("Degree (number of connections)")
plt.ylabel("Number of nodes")
plt.savefig("degree_distribution_acrossFacebookNetwork.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# 5.Betweenness score of nodes
# Detecting users that act as "bridges" in the Facebook network

print("Computing betweenness...")
betw = nx.betweenness_centrality(G)

# Top bridging nodes (structural holes)
top_nodes_with_scores = sorted(betw.items(), key=lambda x: x[1], reverse=True)[:10]
top_nodes = [node for node, score in top_nodes_with_scores]  # only node IDs
print("\nTop 10 structural hole nodes (bridges):")
for node, score in top_nodes_with_scores:
    print(f"Node {node}: betweenness = {score:.5f}")

# Histogram of betweenness values
plt.figure(figsize=(8,5))
plt.hist(betw.values(), bins=40, edgecolor='black')
plt.title("Betweenness Centrality Distribution")
plt.xlabel("Betweenness")
plt.ylabel("Number of Nodes")
plt.tight_layout()
plt.savefig("historgram_betweeness.png", dpi=300, bbox_inches='tight')
plt.show()

# Layout for visualization
pos = nx.spring_layout(G, seed=42)

# Node colors by betweenness
bet_values = [betw[n] for n in G.nodes()]
plt.figure(figsize=(12,10))

# Draw edges first (faint for clarity)
nx.draw_networkx_edges(G, pos, alpha=0.1, edge_color='gray')

# Draw nodes colored by betweenness centrality
nodes = nx.draw_networkx_nodes(
    G, pos,
    node_color=bet_values,
    node_size=50,
    cmap="plasma"
)

# Highlight top structural holes (top 10 betweenness) in red
nx.draw_networkx_nodes(
    G, pos,
    nodelist=top_nodes,
    node_size=200,
    node_color="red",
    edgecolors="black"
)

plt.colorbar(nodes, label="Betweenness Centrality")
plt.title("Structural Holes in Facebook Network\n(Red = Top Betweenness Nodes)")
plt.axis("off")
plt.tight_layout()
plt.savefig("betweeness_wholeNetwork.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# 6. Using the open-triads, create "friend recommendation" system
#If A and C are friends with B, it recommends A as a friend to C

from collections import defaultdict

def recommend_friends_by_triads(G, top_k=5):
    recommendations = defaultdict(lambda: defaultdict(int))

    for B in G.nodes():
        neighbors = list(G.neighbors(B))

        # Check pairs of neighbors: A and C
        for A, C in combinations(neighbors, 2):
            if not G.has_edge(A, C):  
                recommendations[A][C] += 1
                recommendations[C][A] += 1

    final_recs = {}
    for A in recommendations:
        ranked = sorted(recommendations[A].items(), key=lambda x: x[1], reverse=True)
        final_recs[A] = ranked[:top_k]

    return final_recs

friend_recommendations = recommend_friends_by_triads(G)

import random
sample_users = random.sample(list(friend_recommendations.keys()), 5)

for user in sample_users:
    print(f"\nFriend suggestions for user {user}:")
    for rec, score in friend_recommendations[user]:
        print(f" → Suggest {rec} (shared-friends score = {score})")

In [ ]:
# 7.Basic EDA on Facebook Network

print("=== Starting Basic EDA on Facebook Network ===")

# Basic graph info
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
degrees = dict(G.degree())
avg_degree = sum(degrees.values()) / num_nodes
density = nx.density(G)

# Connected components
components = sorted(nx.connected_components(G), key=len, reverse=True)
largest_cc = G.subgraph(components[0]).copy()

# Extra stats on largest connected component
avg_shortest_path = nx.average_shortest_path_length(largest_cc)
diameter = nx.diameter(largest_cc)
avg_clustering = nx.average_clustering(G)

# Print summary
print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")
print(f"Average degree: {avg_degree:.2f}")
print(f"Density: {density:.6f}")
print(f"Number of connected components: {len(components)}")
print(f"Largest connected component: {largest_cc.number_of_nodes()} nodes, {largest_cc.number_of_edges()} edges")
print(f"Average shortest path length (largest CC): {avg_shortest_path:.3f}")
print(f"Diameter (largest CC): {diameter}")
print(f"Average clustering coefficient: {avg_clustering:.3f}")


In [ ]:
# 8.Detecting triadic motifs and assessing the Transistivty 

def triadic_motifs(G: nx.Graph):
    """
    Compute basic 3-node motifs in an undirected simple graph:
    - Number of triangles
    - Number of open triads (wedges)
    - Total number of connected triples
    """
    # Triangles per node: nx.triangles(v) counts triangles incident on v
    triangles_per_node = nx.triangles(G)
    num_triangles = sum(triangles_per_node.values()) // 3  # each triangle counted 3 times

    # Connected triples (2-paths) centered at each node: C(deg(v), 2)
    degrees = dict(G.degree())
    connected_triples = sum(comb(k, 2) for k in degrees.values() if k >= 2)

    # Each triangle contributes 3 triples (one centered on each vertex)
    num_open_triads = connected_triples - 3 * num_triangles

    return {
        "num_triangles": num_triangles,
        "connected_triples": connected_triples,
        "num_open_triads": num_open_triads,
    }

motifs_3 = triadic_motifs(G)

print("\n=== 3-node motifs (ego-Facebook) ===")
print(f"Number of triangles: {motifs_3['num_triangles']}")
print(f"Number of connected triples: {motifs_3['connected_triples']}")
print(f"Number of open triads (wedges): {motifs_3['num_open_triads']}")

# Global transitivity: 3 * triangles / connected_triples
global_transitivity = (
    3 * motifs_3["num_triangles"] / motifs_3["connected_triples"]
    if motifs_3["connected_triples"] > 0
    else 0.0
)
print(f"Global transitivity (triangle fraction): {global_transitivity:.3f}")


In [ ]:
# 9.Detecting Open-Triangles in network

print(f"# Nodes: {G.number_of_nodes()}, # Edges: {G.number_of_edges()}")

# ------------------------------------------------------------------
# 2. Compute open triads per node AND list all open triads explicitly
#    An open triad is (center, u, v) where u, v are neighbors of center,
#    but u and v are NOT directly connected.
# ------------------------------------------------------------------
open_triads_per_node = {}
open_triads = []  # list of (center, u, v)

for center in G.nodes():
    neighbors = list(G.neighbors(center))
    deg = len(neighbors)

    # All unordered pairs of neighbors
    count_open = 0
    for u, v in combinations(neighbors, 2):
        if not G.has_edge(u, v):
            count_open += 1
            open_triads.append((center, u, v))

    open_triads_per_node[center] = count_open

total_open_triads = len(open_triads)
print(f"Total open triads (A-B-C but A≠C, A and C not connected): {total_open_triads}")

# ------------------------------------------------------------------
# 3. Histogram of open triads per node (your original visualization)
# ------------------------------------------------------------------
plt.figure(figsize=(8, 5))
plt.hist(open_triads_per_node.values(), bins=30, edgecolor='black')
plt.title("Open Triads Per Node (Facebook Network)")
plt.xlabel("# Open Triads")
plt.ylabel("# Nodes")
plt.tight_layout()
plt.savefig("open_triads_per_node.png", dpi=300, bbox_inches='tight')
plt.show()

# ------------------------------------------------------------------
# 4. Visualise some open triads in the Facebook network
#    (full Facebook graph is too big, so we sample)
# ------------------------------------------------------------------
if total_open_triads == 0:
    print("No open triads found; nothing to visualize.")
else:
    # Sample up to N open triads for visualization
    N = 40
    sampled_triads = random.sample(open_triads, k=min(N, total_open_triads))

    # Collect all nodes participating in sampled triads
    nodes_in_triads = set()
    centers = set()
    for c, u, v in sampled_triads:
        centers.add(c)
        nodes_in_triads.update([c, u, v])

    # Create an induced subgraph on these nodes
    H = G.subgraph(nodes_in_triads).copy()

    # Layout for visualization
    pos = nx.spring_layout(H, seed=42)

    # Color: centers vs others
    node_colors = []
    for n in H.nodes():
        if n in centers:
            node_colors.append("tab:red")   # center of open triads
        else:
            node_colors.append("tab:blue")  # leaf in open triads

    plt.figure(figsize=(8, 6))

    # Draw existing edges in the subgraph
    nx.draw_networkx_edges(H, pos, alpha=0.4)

    # Draw nodes
    nx.draw_networkx_nodes(
        H, pos,
        node_color=node_colors,
        node_size=80,
        alpha=0.9
    )

    # Optional: add labels for a smaller subset
    if H.number_of_nodes() <= 50:  # avoid clutter
        nx.draw_networkx_labels(
            H, pos,
            font_size=8
        )

    plt.title("Sampled Open Triads in the Facebook Network\n(red = triad centers, blue = leaves)")
    plt.axis("off")
    plt.tight_layout()
    plt.savefig("sampled_open_triads.png", dpi=300, bbox_inches='tight')
    plt.show()

    # ------------------------------------------------------------------
    # 5. (Optional) Draw the *missing* edges of the open triads as dashed lines
    # ------------------------------------------------------------------
    # Build a list of the non-edges (u, v) that make each triad 'open'
    missing_edges = set()
    for c, u, v in sampled_triads:
        # u and v should not be connected in G (by definition), so we add them
        missing_edges.add(tuple(sorted((u, v))))

    # Draw again highlighting missing edges
    plt.figure(figsize=(8, 6))

    # Existing edges first (faint)
    nx.draw_networkx_edges(H, pos, alpha=0.2)

    # Draw missing edges as dashed lines
    for u, v in missing_edges:
        if u in H.nodes() and v in H.nodes():
            x1, y1 = pos[u]
            x2, y2 = pos[v]
            plt.plot([x1, x2], [y1, y2], linestyle="--")

    # Nodes
    nx.draw_networkx_nodes(
        H, pos,
        node_color=node_colors,
        node_size=80,
        alpha=0.9
    )

    if H.number_of_nodes() <= 50:
        nx.draw_networkx_labels(H, pos, font_size=8)

    plt.title("Open Triads in Facebook Network\nDashed lines = missing edges that would close triangles")
    plt.axis("off")
    plt.tight_layout()
    plt.savefig("revised_open_triads.png", dpi=300, bbox_inches='tight')
    plt.show()


In [ ]:
print(f"# Nodes: {G.number_of_nodes()}, # Edges: {G.number_of_edges()}")
# ------------------------------------------------------
# 1. Compute open triads per node (for the whole graph)

# ------------------------------------------------------
# 3. Visualise open triads over the *entire* graph
#    Node color encodes number of open triads it is center of
# ------------------------------------------------------

# Convert values to array for normalization
values = np.array([open_triads_per_node[n] for n in G.nodes()])
vmin, vmax = values.min(), values.max()

# Layout for the full graph (can take some time on big graphs)
pos = nx.spring_layout(G, seed=42)

plt.figure(figsize=(10, 8))

# Draw edges lightly
nx.draw_networkx_edges(G, pos, alpha=0.1, width=0.5)

# Draw nodes colored by #open_triads
nodes = nx.draw_networkx_nodes(
    G, pos,
    node_size=20,
    node_color=values,
    cmap='viridis',
    vmin=vmin,
    vmax=vmax
)

plt.title("Facebook Network Colored by Number of Open Triads per Node")
plt.axis("off")

# Colorbar to interpret the colors
cbar = plt.colorbar(nodes)
cbar.set_label("# Open Triads (as center node)")

plt.tight_layout()
plt.savefig("open_triads_in_whole_network.png", dpi=300, bbox_inches='tight')
plt.show()
